The tutorial below following **Corey Schafer**'s online courses on youtube (https://www.youtube.com/watch?v=ZDa-Z5JzLYM&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc), and I use my own Options-and-Futures class's homework (when studying in NSYSU) as an example. Trying to rewrite the  original code in OOP way. Original code:

In [1]:
# weekly_price = pd.read_csv("Weekly_Price.csv")

#Initial Parameter
W = 1000000
K = 41
Vol = 0.5137
r = 0.01035
T = 20 #Weeks

def Black_Schole_Call(stock_price, K, Vol, r, T):
    if T > 0:
        d1 = (np.log(stock_price / K) + (r + 0.5 * (Vol**2)) * T) / (Vol * np.sqrt(T))
        d2 = d1 - Vol * np.sqrt(T)       
    else:  
        if stock_price >= K:
            d1 = 99999
            d2 = d1 - Vol * np.sqrt(T)
        else:
            d1 = -99999
            d2 = d1 - Vol * np.sqrt(T)
    Call = stock_price * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
    return(Call)


def Delta(stock_price, K, Vol, r, T):   
    if T > 0:
        d1 = (np.log(stock_price / K) + (r + 0.5 * (Vol**2)) * T) / (Vol * np.sqrt(T))   
    else:   
        if stock_price >= K:
            d1 = 99999   
        else:
            d1 = -99999
    return(norm.cdf(d1))


def Calculate_Cost(data):
    data["Interest_expense"] = np.zeros(len(data))
    data["Cumulative_Cost"] = np.zeros(len(data))

    for i in range(len(data)):
        if i == 0:
            data["Cumulative_Cost"][i] = data["Cost_of_Share"][i]
            data["Interest_expense"][i] = data["Cumulative_Cost"][i] * (r * 1/52)
        else:
            data["Cumulative_Cost"][i] = data["Cumulative_Cost"][i - 1] + data["Cost_of_Share"][i] + data["Interest_expense"][i- 1]
            data["Interest_expense"][i] = data["Cumulative_Cost"][i] * (r * 1/52)
            
            
def Calculate_Profit(data):
    Total_Income = (data["Call_Price_BS"][0] * W * (1 + (r * T / 52)))
    
    if (data["Weekly_Price"][len(data) - 1]) >= K:
        Total_Cost = data.Cumulative_Cost[len(data) - 1] + data.Interest_expense[len(data) - 1] - (K * W)
    else:
        Total_Cost = data.Cumulative_Cost[len(data) - 1] + data.Interest_expense[len(data) - 1]
    
    Profit = Total_Income - Total_Cost
    return(Profit)


def Final_Result(K, Vol):
    weekly_price = pd.read_csv("Weekly_Price.csv")
    weekly_price["Expiration"] = pd.Series(range(T,  -1, -1)) / 52 #52 weeks == 1 year
    weekly_price["Call_Price_BS"] = weekly_price.apply(lambda x: Black_Schole_Call(x["Weekly_Price"], K, Vol, r, x["Expiration"]), axis = 1)
    weekly_price["Delta"] = weekly_price.apply(lambda x: Delta(x["Weekly_Price"], K, Vol, r, x["Expiration"]), axis = 1)
    weekly_price["Share_Netural"] = W * weekly_price["Delta"]
    weekly_price["Share_Purchased"] = weekly_price["Share_Netural"].diff(1)
    weekly_price["Share_Purchased"].values[0] = weekly_price["Share_Netural"][0]
    weekly_price["Cost_of_Share"] = weekly_price["Share_Purchased"] * weekly_price["Weekly_Price"]
    Calculate_Cost(weekly_price)
    return(Calculate_Profit(weekly_price))

## Python OOP Tutorial 1: Classes and Instances

> Why OOP?
* Allow us to logically group our data and functions in a way that is easy to reuse and also to build upon if needed.
* Class is basically a blueprint for creating instance, so each instance will contain attribute and method from that class.
* Instance variables contain data that is unique to each instance.
* **Each method within a class automatically takes instance as first argunment(by convention we should call the instance "self").**
* After creating "self" we can specify what other arguments that we want to accept. Within the `__init__`method we should set all instance variables we want. e.g.
```
class Worksheet:
    def __init__(self, W, K, Vol, r, T):
        self.W = W
        self.K = K
        self.Vol = Vol
        self.r = r
        self.T = T
```
the code above is the same as below:
```
class Worksheet:
    pass  
Worksheet.W = W (W is a number)
Worksheet.K = K (K is a number)
.
.
.
```
From the contrast above, we can find that when we create an object, we don't need to assign attribute manually, we just need to create an instance and fill in the variables we want.
* When creating an instance of a class, the "init" method will run automatically. For example:
```
Testing_Worksheet = Worksheet(1, 2, 3, 4, 5)
```
The number pass into Worksheet() is W, K, Vol, r, T sequentially. Note that we don't need to actully put in the "self" variable, because when creating the instance "Testing_Worksheet" itself will be passed in as "self". And then it will set all the attribute in "init" method(W, K, Vol, r, T).